In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import copy 
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from numpy import random as np_random
tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import models

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from operator import add
import random

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
origTra = open('../Datasets/pendigits/pendigits-orig.tra', 'r').read()

In [5]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])
    
len(listOrigTra)

7494

In [6]:
tra = open('../Datasets/pendigits/pendigits.tra', 'r').read()
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])
    
len(listTra)

7494

In [7]:
origTes = open('../Datasets/pendigits/pendigits-orig.tes', 'r').read()

In [8]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])
    
len(listOrigTes)

3498

In [9]:
listOrigTes = [str(int(i)+30) for i in listOrigTes]

In [10]:
tes = open('../Datasets/pendigits/pendigits.tes', 'r').read()
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])
    
len(listTes)

3498

In [11]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [12]:
df_tra = pd.read_csv('../Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [13]:
df_tra_w = df_tra.copy()
df_tra_w['writer'] = listOrigTra
df_tra_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [14]:
df_tes = pd.read_csv('../Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [15]:
df_tes.to_csv('TEST.csv')

In [16]:
df_tes_w = df_tes.copy()
df_tes_w['writer'] = listOrigTes
df_tes_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8,36
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8,42
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8,35
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9,37
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9,35


In [17]:
df = pd.concat([df_tra_w,df_tes_w])
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [18]:
d0 = df_tra_w.loc[df_tra_w.digit==0].shape[0]
d1 = df_tra_w.loc[df_tra_w.digit==1].shape[0]
d2 = df_tra_w.loc[df_tra_w.digit==2].shape[0]
d3 = df_tra_w.loc[df_tra_w.digit==3].shape[0]
d4 = df_tra_w.loc[df_tra_w.digit==4].shape[0]
d5 = df_tra_w.loc[df_tra_w.digit==5].shape[0]
d6 = df_tra_w.loc[df_tra_w.digit==6].shape[0]
d7 = df_tra_w.loc[df_tra_w.digit==7].shape[0]
d8 = df_tra_w.loc[df_tra_w.digit==8].shape[0]
d9 = df_tra_w.loc[df_tra_w.digit==9].shape[0]

print(d0)
print(d1)
print(d2)
print(d3)
print(d4)
print(d5)
print(d6)
print(d7)
print(d8)
print(d9)

D={
        0:d0,
        1:d1,
        2:d2,
        3:d3,
        4:d4,
        5:d5,
        6:d6,
        7:d7,
        8:d8,
        9:d9,
    
}

780
779
780
719
780
720
720
778
719
719


In [19]:
pickle.dump(D,open('COUNT','wb'))

In [20]:
def getClients(data):
    arr = []
    no_of_writers = len(set(data['writer']))
    for i in range(no_of_writers):
        arr.append([])
    for index, row in data.iterrows():
        rowA = row.to_numpy()
        writer = int(rowA[-1])
        data = rowA[:17]
        arr[writer-1].append(data)
    return arr

In [21]:
def get_batches(HOLDER):
    federated_data = []
    for i in HOLDER:
        client = []
        for index in range(len(i)//20):
            X= []
            Y= []
            for elements in i[index*20:index*20+20]:
                X.append(np.array([np.float32(elements[0]),
                                   np.float32(elements[1]),
                                   np.float32(elements[2]),
                                   np.float32(elements[3]),
                                   np.float32(elements[4]),
                                   np.float32(elements[5]),
                                   np.float32(elements[6]),
                                   np.float32(elements[7]),
                                   np.float32(elements[8]),
                                   np.float32(elements[9]),
                                   np.float32(elements[10]),
                                   np.float32(elements[11]),
                                   np.float32(elements[12]),
                                   np.float32(elements[13]),
                                   np.float32(elements[14]),
                                   np.float32(elements[15])]))
                Y.append(np.array(np.int32(elements[-1])))
            client.append({
                'x':np.array(X),
                'y':np.array(Y)
            })
        federated_data.append(client)
    return federated_data

In [22]:
# federated_train_data = getClients(df_tra_w)
# federated_test_data = getClients(df_tes_w)

In [23]:
# federated_train_data =  get_batches(train)
# federated_validation_data =  get_batches(validation)
# federated_test_data =  get_batches(test)

In [24]:
def getModel():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(10,activation="sigmoid")
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [25]:
import numpy as np
from collections import Counter

def getweight(df):
    LBL = Counter(df.digit)
    maen_arr = np.mean(list(map(lambda x:x[1]/D[x[0]],list(LBL.items()))))
    return maen_arr

In [26]:
writer_data = df_tra_w.groupby(df_tra_w.writer)

In [27]:
import pickle
from tqdm.notebook import tqdm
GOD = []

In [28]:
for i in tqdm(range(1)):
    d ={}
    for i in range(1,31):
        d[i]=writer_data.get_group('{}'.format(i)).sample(50)

#     pickle.dump(d,open('R1','wb'))

    X = []
    for i in range(10):
        d = {}
        for i in range(1,31):
            d[i]=writer_data.get_group('{}'.format(i)).sample(50)
        X.append(d)
        
    GOD.append({
        'fs':d,
        'snd':X
    })

In [29]:
pickle.dump(GOD,open('GOD.pkl','wb'))

In [30]:
df_sample_test = pd.concat([df_tra, df_tes], axis=0).sample(25)
pickle.dump(df_sample_test, open('df_sample_test','wb'))

In [31]:
df_sample_test

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
4582,70,97,28,78,65,53,57,9,0,0,13,42,51,86,100,100,5
6482,87,100,48,100,17,67,6,31,38,0,100,12,52,29,0,4,6
2030,100,99,52,79,0,70,26,100,80,100,81,65,57,32,35,0,9
2675,55,68,71,40,100,6,5,0,14,35,66,68,89,100,0,83,8
2582,38,100,16,86,0,51,55,45,92,70,100,72,95,36,95,0,4
3107,96,100,60,82,18,54,0,33,63,26,99,49,94,34,100,0,4
6609,71,100,47,87,21,60,0,33,24,12,97,26,100,26,76,0,4
4245,100,94,51,83,0,76,40,100,80,96,58,61,29,30,8,0,9
651,0,44,30,61,67,80,100,100,96,80,83,53,68,26,62,0,1
6724,0,75,15,53,77,22,33,0,20,34,72,67,100,100,5,89,8


In [32]:
def get_digit_count_dict():
    d = dict()
    for i in range(1,11):
        d[i] = 0
    return d

In [33]:
def make_digit_count_dict(digit_data,digit_counts):
    for i in range(10):
        digit_counts[i+1]+=digit_data.get_group(i).shape[0]
    return digit_counts

In [34]:
def get_probabilies(client):
    digit_counts = get_digit_count_dict()
    data = writer_data.get_group('{}'.format(client))
    digit_data = data.groupby(data.digit)
    counts = make_digit_count_dict(digit_data,digit_counts)  
    total = sum(counts.values(), 0.0)
    return {key: counts[key] / total for key in counts.keys()}

In [35]:
def draw_categorical(client, n):
    r = 1
    z = []
    probabilies = get_probabilies(client)
    print(client)
    print(probabilies)
    for i in range(1,11):
        prob = probabilies[i]
        print('nadeeja')
#         print(prob)
        p = prob/r
        print(prob)
        print(r)
        print('nadeeja')
        v = np_random.binomial(n, p)
        for j in range(v):
            z.append(i)
        n = n - v
        r = r - prob
    random.shuffle(z)
    return z

In [36]:
for client in range(1,31):
    draw_categorical(client,25)

1
{1: 0.10441767068273092, 2: 0.10441767068273092, 3: 0.10441767068273092, 4: 0.0963855421686747, 5: 0.10441767068273092, 6: 0.0963855421686747, 7: 0.0963855421686747, 8: 0.10441767068273092, 9: 0.0963855421686747, 10: 0.09236947791164658}
nadeeja
0.10441767068273092
1
nadeeja
nadeeja
0.10441767068273092
0.8955823293172691
nadeeja
nadeeja
0.10441767068273092
0.7911646586345382
nadeeja
nadeeja
0.0963855421686747
0.6867469879518073
nadeeja
nadeeja
0.10441767068273092
0.5903614457831327
nadeeja
nadeeja
0.0963855421686747
0.48594377510040176
nadeeja
nadeeja
0.0963855421686747
0.3895582329317271
nadeeja
nadeeja
0.10441767068273092
0.2931726907630524
nadeeja
nadeeja
0.0963855421686747
0.1887550200803215
nadeeja
nadeeja
0.09236947791164658
0.09236947791164679
nadeeja
2
{1: 0.104, 2: 0.104, 3: 0.104, 4: 0.096, 5: 0.104, 6: 0.096, 7: 0.096, 8: 0.104, 9: 0.096, 10: 0.096}
nadeeja
0.104
1
nadeeja
nadeeja
0.104
0.896
nadeeja
nadeeja
0.104
0.792
nadeeja
nadeeja
0.096
0.6880000000000001
nadeeja
nade

ValueError: p < 0, p > 1 or p is NaN

In [37]:
0.09716599190283401/0.09716599190283394

1.0000000000000007

In [38]:
0.096/0.09600000000000017

0.9999999999999982

In [51]:
d=writer_data.get_group('17')
digit_data = d.groupby(d.digit)
s=d.shape[0]
for i in range(10):
    print(digit_data.get_group(i).shape[0]/s)

0.10526315789473684
0.10526315789473684
0.10526315789473684
0.09716599190283401
0.10526315789473684
0.09716599190283401
0.09716599190283401
0.09716599190283401
0.0931174089068826
0.09716599190283401


In [52]:
{1: 0.10526315789473684, 2: 0.10526315789473684, 3: 0.10526315789473684, 4: 0.09716599190283401, 5: 0.10526315789473684, 6: 0.09716599190283401, 7: 0.09716599190283401, 8: 0.09716599190283401, 9: 0.0931174089068826, 10: 0.09716599190283401}

{1: 0.10526315789473684,
 2: 0.10526315789473684,
 3: 0.10526315789473684,
 4: 0.09716599190283401,
 5: 0.10526315789473684,
 6: 0.09716599190283401,
 7: 0.09716599190283401,
 8: 0.09716599190283401,
 9: 0.0931174089068826,
 10: 0.09716599190283401}

In [53]:
0.09600000000000017-0.096

1.6653345369377348e-16